In [1]:
# import load_dotenv
from dotenv import load_dotenv

In [2]:
# load env
load_dotenv()

True

In [4]:
from langchain_core.tools import tool

@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

# print out
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [19]:
from langchain_core.tools import tool

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [20]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [21]:
# OpenAI
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [22]:
# use bind_tools to pass the definition of our tool in as part of each call to the model
# so that the model can invoke the tool when appropriate
tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)

In [23]:
# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output
msg = llm_with_tools.invoke("whats 5 times forty two")
print(type(msg))
print(msg)
print(msg.tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
content='' additional_kwargs={'tool_calls': [{'id': 'call_pjgAbGlkJFQWL5TyL5kXDSs8', 'function': {'arguments': '{"first_int":5,"second_int":42}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 149, 'total_tokens': 168}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-1c662293-8a91-466d-a2ab-21d87118acef-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_pjgAbGlkJFQWL5TyL5kXDSs8'}]
[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_pjgAbGlkJFQWL5TyL5kXDSs8'}]


92

In [24]:
msg = llm_with_tools.invoke("what is a neural network?")
print(type(msg))
print(msg)
print(msg.tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
content="A neural network is a type of artificial intelligence (AI) algorithm modeled after the human brain's neural network structure. It consists of interconnected nodes, called neurons, that work together to process and analyze complex data. Neural networks are capable of learning from data, identifying patterns, and making decisions without being explicitly programmed.\n\nNeural networks are commonly used in various applications, such as image recognition, natural language processing, speech recognition, and many other fields where complex pattern recognition and decision-making are required. The structure of a neural network typically includes input nodes, hidden layers, and output nodes, with each layer performing specific computations to transform the input data into meaningful output.\n\nOverall, neural networks are a powerful tool in machine learning and AI, enabling machines to perform tasks that mimic human cognitive functions."

In [26]:
msg = llm_with_tools.invoke("What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven")
print(type(msg))
print(msg)
print(msg.tool_calls)

<class 'langchain_core.messages.ai.AIMessage'>
content='' additional_kwargs={'tool_calls': [{'id': 'call_GRHcyNYKaSIV7VQUX0z7AHfG', 'function': {'arguments': '{"first_int": 23, "second_int": 7}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_r55Xw6sASz7pRYruSS3ImTko', 'function': {'arguments': '{"first_int": 5, "second_int": 18}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_eDqGG4oHEyk7eAYr481BMpGq', 'function': {'arguments': '{"first_int": 1000000, "second_int": 1000000000}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_j1XVlK6AtLrrT58V5tN4ry0R', 'function': {'arguments': '{"base": 37, "exponent": 3}', 'name': 'exponentiate'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 168, 'total_tokens': 265}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-97714222-5923-4aca-a1d7-81f1c2822161-0' tool_calls=[{'name': 'multiply', 'args': 

In [25]:
tools_chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
tools_chain.invoke("What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven")

161

In [33]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

tool_map = {tool.name: tool for tool in tools}

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    result = {}
    if len(msg.content) > 0:
        result['content'] = msg.content
        return result
        
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    result['tools'] = tool_calls
    return result

In [34]:
tools_chain = llm_with_tools | call_tools

In [35]:
result = tools_chain.invoke(
    "What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven"
)

In [36]:
result

{'tools': [{'name': 'multiply',
   'args': {'first_int': 23, 'second_int': 7},
   'id': 'call_jXu58lZvatgMlOceXPTBH5mm',
   'output': 161},
  {'name': 'multiply',
   'args': {'first_int': 5, 'second_int': 18},
   'id': 'call_efbnVtcRvCGA75OmHK1wAItW',
   'output': 90},
  {'name': 'add',
   'args': {'first_int': 1000000, 'second_int': 1000000000},
   'id': 'call_hQdKygNh1MO7OnUdAHD8xHWr',
   'output': 1001000000},
  {'name': 'exponentiate',
   'args': {'base': 37, 'exponent': 3},
   'id': 'call_e8MSG7ydnF9pf4rBv6VqjGVf',
   'output': 50653}]}

In [37]:
result = tools_chain.invoke(
    "What's deep learning"
)
print(result)

{'content': 'Deep learning is a subset of machine learning that uses artificial neural networks to model and solve complex problems. It is called "deep" learning because it involves multiple layers of interconnected neurons that process and learn from data. These neural networks are capable of automatically learning representations of data through a process of hierarchical feature learning.\n\nDeep learning has been particularly successful in tasks such as image recognition, speech recognition, natural language processing, and more. It has revolutionized many fields and enabled advancements in areas like autonomous driving, healthcare, and robotics.'}


In [38]:
result = tools_chain.invoke("whats 5 times forty two")
print(result)

{'tools': [{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_rhyNOUCviDSYEYoffPVEaXdJ', 'output': 210}]}
